In [1]:
import os
import glob
import matplotlib.pylab as plt
import matplotlib as mpl
import numpy as np
from skimage.io import imread

In [2]:
class Cases:
    
    def __init__(self, path='../data/preprocess/'):
        self.path = path
        self.cases = [f.path[19:] for f in os.scandir(path) if f.is_dir()]
        self.before_dict = self._fill_dicts()
        self.after_dict = self._fill_dicts(kind='AFTER')
        
    def _fill_dicts(self, kind='BEFORE') -> dict:
        cdict = {}
        for case in self.cases:
            lst = [imread(file) for file in glob.glob(self.path + case + "/" + kind + "/*.JPG")]
            cdict[case] = lst
        return cdict
            
    @property
    def before(self) -> dict:
        return self.before_dict
    
    @property
    def after(self) -> dict:
        return self.after_dict